## VacationPy

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from api_keys import g_key

## Create DataFrom for WeatherPY

In [2]:
weather_data = pd.read_csv("../WeatherPy/output_data/cities.csv")
weather_data

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Torbay,20,CA,1619033614,57,47.6666,-52.7314,51.01,18.41
1,Vaini,40,TO,1619033812,94,-21.2000,-175.2000,73.40,3.44
2,Leningradskiy,33,RU,1619033812,67,69.3833,178.4167,27.19,5.97
3,Puerto Ayora,75,EC,1619033813,78,-0.7393,-90.3518,80.60,11.50
4,George Town,40,MY,1619033583,89,5.4112,100.3354,80.60,2.30
...,...,...,...,...,...,...,...,...,...
537,Santiago,40,CL,1619033924,35,-33.4569,-70.6483,75.00,8.05
538,Shimoda,5,JP,1619033959,79,34.6667,138.9500,52.00,3.00
539,Benguela,27,AO,1619033959,85,-12.5763,13.4055,81.09,4.72
540,Greifswald,1,DE,1619033959,68,54.1000,13.3833,44.01,1.01


### Create a Humidity Heatmap

In [3]:
# API Key for gmaps
gmaps.configure(api_key=g_key)

# Set latitude and longitude
locations = weather_data[["Lat", "Lng"]]

# Set Humidity
humidity = weather_data["Humidity"]

In [5]:
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -8.0), zoom_level=2)
max_intensity = np.max(humidity)

# heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Narrow down DataFrame for ideal weather conditions

In [6]:
# Narrow down the cities with wind speed less than 10 mph, cloudiness equals to 0 and max temp between 70 and 85
narrow_city_df = weather_data.loc[(weather_data["Wind Speed"] <= 10) & (weather_data["Cloudiness"] == 0) & \
                                   (weather_data["Max Temp"] >= 70) & (weather_data["Max Temp"] <= 85)].dropna()

narrow_city_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
93,Sittwe,0,MM,1619033833,85,20.1500,92.9000,79.47,5.82
95,Shwebo,0,MM,1619033834,26,22.5667,95.7000,82.78,4.27
183,Pitimbu,0,BR,1619033854,70,-7.4706,-34.8086,84.20,9.22
227,Hong Kong,0,HK,1619033614,83,22.2855,114.1577,73.99,6.33
279,Olinda,0,BR,1619033899,65,-8.0089,-34.8553,82.40,9.22
339,Doha,0,QA,1619033850,65,25.2867,51.5333,82.00,3.44
376,Buenos Aires,0,AR,1619033627,73,-34.6132,-58.3772,78.01,9.22
397,Araxá,0,BR,1619033925,32,-19.5933,-46.9406,80.60,4.61
429,Fīrozpur Jhirka,0,IN,1619033934,41,27.8000,76.9500,75.45,5.59


### Hotel Map

In [7]:
# hotel data frame
hotel_df = narrow_city_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add  "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

hotel_df

,City,Country,Lat,Lng,Hotel Name
93,Sittwe,MM,20.1500,92.9000,
95,Shwebo,MM,22.5667,95.7000,
183,Pitimbu,BR,-7.4706,-34.8086,
227,Hong Kong,HK,22.2855,114.1577,
279,Olinda,BR,-8.0089,-34.8553,
339,Doha,QA,25.2867,51.5333,
376,Buenos Aires,AR,-34.6132,-58.3772,
397,Araxá,BR,-19.5933,-46.9406,
429,Fīrozpur Jhirka,IN,27.8000,76.9500,


Find first hotel located within 5000 meters of the coordinates

In [8]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [9]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add params dict
    params["location"] = f"{lat},{lng}"

    # url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if no hotel, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
   
   
# Print end of search once searching is completed
print("-------End of Search-------")

Retrieving Results for Index 93: Sittwe.
Closest hotel in Sittwe is Hotel Memory.
------------
Retrieving Results for Index 95: Shwebo.
Closest hotel in Shwebo is Sann Tin Hotel.
------------
Retrieving Results for Index 183: Pitimbu.
Closest hotel in Pitimbu is Asenza Beach Resort.
------------
Retrieving Results for Index 227: Hong Kong.
Closest hotel in Hong Kong is The Murray, Hong Kong, a Niccolo Hotel.
------------
Retrieving Results for Index 279: Olinda.
Closest hotel in Olinda is Hotel Mar Aberto Olinda.
------------
Retrieving Results for Index 339: Doha.
Closest hotel in Doha is The Curve Hotel.
------------
Retrieving Results for Index 376: Buenos Aires.
Closest hotel in Buenos Aires is HTL City Baires.
------------
Retrieving Results for Index 397: Araxá.
Closest hotel in Araxá is Hotel Dona Beja.
------------
Retrieving Results for Index 429: Fīrozpur Jhirka.
Missing field/result... skipping.
------------
-------End of Search-------


In [10]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
93,Sittwe,MM,20.1500,92.9000,Hotel Memory
95,Shwebo,MM,22.5667,95.7000,Sann Tin Hotel
183,Pitimbu,BR,-7.4706,-34.8086,Asenza Beach Resort
227,Hong Kong,HK,22.2855,114.1577,"The Murray, Hong Kong, a Niccolo Hotel"
279,Olinda,BR,-8.0089,-34.8553,Hotel Mar Aberto Olinda
339,Doha,QA,25.2867,51.5333,The Curve Hotel
376,Buenos Aires,AR,-34.6132,-58.3772,HTL City Baires
397,Araxá,BR,-19.5933,-46.9406,Hotel Dona Beja
429,Fīrozpur Jhirka,IN,27.8000,76.9500,


### Add pin points to Heat Map

In [11]:
# add the hotel marks to the heatmap
info_box = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

hotel_info = [info_box.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [15]:
# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))